In [1]:
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm

Para crear un juego de cuatro en raya en un tablero de 4x4 realice cambios en la clase Board, Game
y en la logica de entrenamiento y juego

In [2]:
#clase para iniciar el tablero como una matriz de 4x4 llena de ceros
#modificando la inicialización del tablero y las funciones que dependen del tamaño del tablero.
class Board():
    def __init__(self):
        self.state = np.zeros((4,4))

    #Devuelve una lista de movimientos válidos las celdas vacias
    def valid_moves(self):
        return [(i, j) for j in range(4) for i in range(4) if self.state[i, j] == 0]

    #Actualiza el tablero con el símbolo del jugador en la posición especificada si está vacía
    def update(self, symbol, row, col):
        if self.state[row, col] == 0:
            self.state[row, col] = symbol
        else:
            raise ValueError("Movimiento ilegal !")

    #Comprueba si el juego ha terminado si hay ganador empate o se perdio
    #Ajustando esta funcion para comprobar filas, columnas y diagonales del tablero 
    def is_game_over(self):
        # Comprobar filas y columnas
        for i in range(4):
            if abs(self.state[i, :].sum()) == 4:
                return self.state[i, 0]
            if abs(self.state[:, i].sum()) == 4:
                return self.state[0, i]
        # Comprobar diagonales
        if abs(self.state.trace()) == 4 or abs(np.fliplr(self.state).trace()) == 4:
            return self.state[0, 0]
        # Empate
        if len(self.valid_moves()) == 0:
            return 0
        # Seguir jugando
        return None

    #Reinicia el tablero a su estado inicial con atriz de ceros
    def reset(self):
        self.state = np.zeros((4,4))

In [3]:
#clase para iniciar el juego con dos jugadores y un tablero nuevo.
class Game():
    def __init__(self, player1, player2):
        player1.symbol = 1
        player2.symbol = -1
        self.players = [player1, player2]
        self.board = Board()

    #Realiza varias partidas entre los dos agentes para entrenarlos.
    def selfplay(self, rounds=100):
        wins = [0, 0]
        for _ in tqdm(range(rounds)):
            self.board.reset()
            for player in self.players:
                player.reset()
            game_over = False
            while not game_over:
                for player in self.players:
                    action = player.move(self.board)
                    self.board.update(player.symbol, action[0], action[1])
                    if (result := self.board.is_game_over()) is not None:
                        game_over = True
                        break
            self.reward()
            for ix, player in enumerate(self.players):
                if result == player.symbol:
                    wins[ix] += 1
        return wins

    #Para asignar recompensas a los jugadores al final de cada partida
    def reward(self):
        winner = self.board.is_game_over()
        if winner == 0: # Empate
            for player in self.players:
                player.reward(0.5)
        else: # Le damos 1 recompensa al jugador que gana
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1)
                else:
                    player.reward(0)

In [4]:
#Inicializa el agente con una función de valor.
class Agent():
    def __init__(self, alpha=0.5, prob_exp=0.5):
        self.value_function = {} # Tabla con pares estado -> valor
        self.alpha = alpha         # tasa de aprendizaje (alpha)
        self.positions = []       # Guardamos todas las posiciones visitadas
        self.prob_exp = prob_exp   # Probabilidad de explorar

    #Resetea las posiciones visitadas
    def reset(self):
        self.positions = []

    #Realiza un movimiento basado en la exploración (movimiento aleatorio) o explotación (mejor movimiento conocido)
    def move(self, board, explore=True):
        valid_moves = board.valid_moves()
        # Exploración
        if explore and np.random.uniform(0, 1) < self.prob_exp:
            # Vamos a una posición aleatoria
            ix = np.random.choice(len(valid_moves))
            return valid_moves[ix]
        # Explotación
        # Vamos a la posición con más valor
        max_value = -1000
        for row, col in valid_moves:
            next_board = board.state.copy()
            next_board[row, col] = self.symbol
            next_state = str(next_board.reshape(4*4))
            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state)
            if value >= max_value:
                max_value = value
                best_row, best_col = row, col
        return best_row, best_col
    
    #Guarda el estado actual del tablero en la lista de posiciones visitadas
    def update(self, board):
        self.positions.append(str(board.state.reshape(4*4)))

    #Actualiza la función de valor de cada estado visitado en función de la recompensa obtenida al final de la partida
    def reward(self, reward):
        for p in reversed(self.positions):
            if self.value_function.get(p) is None:
                self.value_function[p] = 0
            self.value_function[p] += self.alpha * (reward - self.value_function[p])
            reward = self.value_function[p]

In [5]:
#Entrena dos agentes jugando entre ellos
agent1 = Agent(prob_exp=0.5)
agent2 = Agent()

game = Game(agent1, agent2)

game.selfplay(30000)

100%|██████████| 30000/30000 [06:52<00:00, 72.72it/s]


[8369, 6758]

In [6]:
#Guarda la funcion de valor del primer agente en un archivo utilizando pickle.
funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})

tabla

,estado,valor


In [7]:
with open('agente.pickle', 'wb') as handle:
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
# Crear una nueva instancia de juego con los agentes ya entrenados
game = Game(agent1, agent2)

# Función para mostrar el estado del tablero
def print_board(board):
    for row in board.state:
        print(" | ".join(["X" if cell == 1 else "O" if cell == -1 else "-" for cell in row]))
        print("-" * 19)

# Jugar contra el agente
while True:
    print_board(game.board)
    row = int(input("Ingrese la fila (0-3): "))
    col = int(input("Ingrese la columna (0-3): "))
    if (row, col) in game.board.valid_moves():
        game.board.update(1, row, col)
        if (result := game.board.is_game_over()) is not None:
            print_board(game.board)
            if result == 1:
                print("¡Has ganado!")
            elif result == -1:
                print("¡Has perdido!")
            else:
                print("¡Empate!")
            break
        action = agent2.move(game.board)
        game.board.update(-1, action[0], action[1])
        if (result := game.board.is_game_over()) is not None:
            print_board(game.board)
            if result == 1:
                print("¡Has ganado!")
            elif result == -1:
                print("¡Has perdido!")
            else:
                print("¡Empate!")
            break
    else:
        print("Movimiento inválido, intenta de nuevo.")


- | - | - | -
-------------------
- | - | - | -
-------------------
- | - | - | -
-------------------
- | - | - | -
-------------------
- | - | - | -
-------------------
X | - | - | -
-------------------
- | - | - | -
-------------------
- | - | - | O
-------------------
- | - | - | -
-------------------
X | X | - | -
-------------------
- | - | - | -
-------------------
O | - | - | O
-------------------
O | - | - | -
-------------------
X | X | X | -
-------------------
- | - | - | -
-------------------
O | - | - | O
-------------------
O | - | - | -
-------------------
X | X | X | X
-------------------
- | - | - | -
-------------------
O | - | - | O
-------------------
¡Has ganado!
